In [1]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Activation, Dropout, Flatten, ActivityRegularization
from keras.layers.core import Activation
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras import losses

from PIL import Image
import glob

import tensorflow as tf
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
# Function to create model, required for KerasClassifier
def create_model(learn=0.01,lost='categorical_crossentropy'):
    # create model
    
    model = Sequential()

    model.add(Conv2D(200, (5, 5), input_shape=(200, 200, 1), padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(100))
    model.add(Dropout(0.1))
    model.add(AveragePooling2D(pool_size=(3,3)))

    model.add(Conv2D(200, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(50))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size = (3,3)))

    model.add(Conv2D(200, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dense(25))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size = (3,3)))

    model.add(Flatten())
    model.add(Dense(6))
    model.add(Dropout(0.1))
    model.add(Activation('softmax'))
    
    # Compile model
    sgd = optimizers.SGD(lr=learn, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss=lost, metrics=['accuracy'])
    
    return model

In [3]:
with tf.device("/gpu:0"):
    train_test_datagen = ImageDataGenerator(fill_mode='constant', 
                                        data_format='channels_last', 
                                        rescale=1./255)

    train = train_test_datagen.flow_from_directory(
            '/home/ubuntu/project_two/new_image_directory',
            target_size=(200,200),
            batch_size=25,
            class_mode='categorical',
            color_mode='grayscale')
    hist = []

    tbCallBack = TensorBoard(log_dir='./TbGraph', histogram_freq=0, write_graph=True, write_images=True)
    
    for i in [0.3, 0.2, 0.1, 0.07, 0.05]:
        for j in ['mse','mae','mape','msle','kld','cosine','categorical_crossentropy']:
            model = None
            model = create_model(i,j)

            history = model.fit_generator(
                    train,
                    steps_per_epoch=25,
                    epochs=5,
                    validation_data=train,
                    validation_steps=25,
                    callbacks=[tbCallBack])
            hist.append(history)

Found 9600 images belonging to 6 classes.
Epoch 1/5
25/25 [==============================] - 120s - loss: 0.1393 - acc: 0.1696 - val_loss: 0.1390 - val_acc: 0.1568
Epoch 2/5
25/25 [==============================] - 121s - loss: 0.1390 - acc: 0.1712 - val_loss: 0.1390 - val_acc: 0.1744
Epoch 3/5
25/25 [==============================] - 121s - loss: 0.1388 - acc: 0.2032 - val_loss: 0.1393 - val_acc: 0.1728
Epoch 4/5
25/25 [==============================] - 121s - loss: 0.1384 - acc: 0.1776 - val_loss: 0.1398 - val_acc: 0.1680
Epoch 5/5
25/25 [==============================] - 121s - loss: 0.1381 - acc: 0.2144 - val_loss: 0.1389 - val_acc: 0.2064
Epoch 1/5
25/25 [==============================] - 121s - loss: 0.2833 - acc: 0.1600 - val_loss: 0.2757 - val_acc: 0.1728
Epoch 2/5
25/25 [==============================] - 121s - loss: 0.2776 - acc: 0.1648 - val_loss: 0.2875 - val_acc: 0.1376
Epoch 3/5
25/25 [==============================] - 120s - loss: 0.2704 - acc: 0.1904 - val_loss: 0.2800 

25/25 [==============================] - 118s - loss: -0.0680 - acc: 0.1744 - val_loss: -0.0681 - val_acc: 0.1936
Epoch 5/5
25/25 [==============================] - 118s - loss: -0.0684 - acc: 0.1888 - val_loss: -0.0681 - val_acc: 0.2096
Epoch 1/5
25/25 [==============================] - 118s - loss: 1.8056 - acc: 0.1792 - val_loss: 1.8421 - val_acc: 0.1824
Epoch 2/5
25/25 [==============================] - 118s - loss: 1.8229 - acc: 0.1792 - val_loss: 1.8287 - val_acc: 0.1504
Epoch 3/5
25/25 [==============================] - 118s - loss: 1.8351 - acc: 0.1648 - val_loss: 1.9068 - val_acc: 0.1664
Epoch 4/5
25/25 [==============================] - 118s - loss: 1.8366 - acc: 0.1744 - val_loss: 1.8909 - val_acc: 0.1616
Epoch 5/5
25/25 [==============================] - 118s - loss: 1.8340 - acc: 0.1536 - val_loss: 1.8517 - val_acc: 0.1680
Epoch 1/5
25/25 [==============================] - 119s - loss: 0.1390 - acc: 0.1360 - val_loss: 0.1389 - val_acc: 0.1440
Epoch 2/5
25/25 [=============